In [13]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch

In [12]:
import mthree

In [4]:
instance = 'ibm-q-internal/performance/reservations'
service = QiskitRuntimeService(instance=instance)
cairo = service.backend("ibm_cairo")

In [5]:
## Create circuits

In [7]:
num_qubits = 4
qc0 = QuantumCircuit(num_qubits)
qc0.h(0)
for q in range(1, num_qubits):
    qc0.cx(0, q)
    qc0.cx(0, q)
    qc0.cx(0, q)
qc0.measure_all()

In [8]:
num_qubits = 5
qc1 = QuantumCircuit(num_qubits)
qc1.h(0)
for q in range(1, num_qubits):
    qc1.cx(0, q)
    qc1.cx(0, q)
    qc1.cx(0, q)
qc1.measure_all()

In [9]:
num_qubits = 6
qc2 = QuantumCircuit(num_qubits)
qc2.h(0)
for q in range(1, num_qubits):
    qc2.cx(0, q)
    qc2.cx(0, q)
    qc2.cx(0, q)
qc2.measure_all()

## Transpilation

In [11]:
trans_qcs = transpile([qc0, qc1, qc2], backend=cairo, optimization_level=3)

## Run

In [14]:
m3_mitigators = []
jobs = []
with Batch(backend=cairo):
    sampler = Sampler()
    
    job0 = sampler.run([trans_qcs[0]])
    jobs.append(job0)
    # same as before. just supply `sampler` instead of `backend`
    m3_mit = mthree.M3Mitigation(sampler)
    final_layout = trans_qcs[0].layout.final_index_layout(filter_ancillas=True)
    # run the measurement calibration
    m3_mit.cals_from_system(qubits=final_layout)
    # save the M3mitigation object to apply correction after all HW runs are done
    # applying correction later for non-iterative jobs may save time
    m3_mitigators.append(m3_mit)

    job1 = sampler.run([trans_qcs[1]])
    jobs.append(job1)
    m3_mit = mthree.M3Mitigation(sampler)
    final_layout = trans_qcs[1].layout.final_index_layout(filter_ancillas=True)
    m3_mit.cals_from_system(qubits=final_layout)
    m3_mitigators.append(m3_mit)

    job2 = sampler.run([trans_qcs[2]])
    jobs.append(job2)
    m3_mit = mthree.M3Mitigation(sampler)
    final_layout = trans_qcs[2].layout.final_index_layout(filter_ancillas=True)
    m3_mit.cals_from_system(qubits=final_layout)
    m3_mitigators.append(m3_mit)

In [22]:
for idx, job in enumerate(jobs):
    raw_counts = job.result()[0].data.meas.get_counts()
    print(f'Job {idx} raw counts {raw_counts}\n')

    quasi_prob = m3_mitigators[idx].apply_correction(
        counts=raw_counts,
        qubits=trans_qcs[idx].layout.final_index_layout(filter_ancillas=True),
        distance=6, # to save time for circuits with many qubits
    )
    print(f'mitigated quasi prob {quasi_prob}\n\n')
    print("#"*30)

Job 0 raw counts {'0000': 1831, '1111': 1709, '0001': 40, '1101': 128, '1011': 66, '0111': 40, '0100': 58, '0110': 33, '0101': 12, '1001': 20, '1100': 5, '0010': 56, '1110': 51, '1000': 36, '0011': 8, '1010': 3}

mitigated quasi prob {'0000': 0.49308399664658237, '0001': -0.009118443107739814, '0010': 0.007029280360616579, '0011': 0.001397553070229798, '0100': 0.002867859059716595, '0101': 0.0021143916381924917, '0110': 0.008416825829240204, '0111': 0.0034392137373366704, '1000': 0.00038828200792753726, '1001': 0.0039863408864391275, '1010': -2.4770310691775155e-05, '1011': 0.007950006702004719, '1100': 1.928917758149729e-06, '1101': 0.010175088722610482, '1110': -0.0013843942055908165, '1111': 0.46967684004536797}


##############################
Job 1 raw counts {'00000': 1099, '11111': 933, '10111': 55, '00110': 14, '01111': 16, '11011': 65, '11101': 54, '01101': 6, '11110': 21, '01000': 12, '00100': 44, '00010': 53, '10011': 9, '00001': 27, '10000': 19, '10010': 2, '10001': 4, '110